使用LoFTR模型直接推理，Public LB 0.726；

对LoFTR进行调参（分辨率/keypoints数量/threshold等），Public LB 0.767；

使用SuperGlue并调参，Public LB : 0.710；

使用QTA并调参，Public LB : 0.799；

使用DKM并调参，Public LB : 0.600；

对四个模型结果进行融合，Public LB : 0.830+；

对cv2.findFundamentalMat进行调参，Public LB : 0.840+；
 

# 安装/导入 model lib

In [ ]:
import os
import sys

!pip install ../input/pywheels/loguru-0.5.3-py3-none-any.whl
!pip install ../input/pywheels/einops-0.4.1-py3-none-any.whl
!pip install ../input/pywheels/timm-0.4.12-py3-none-any.whl

#kornia
!pip install /kaggle/input/kornia-loftr/kornia-0.6.4-py2.py3-none-any.whl
!pip install /kaggle/input/kornia-loftr/kornia_moons-0.1.9-py3-none-any.whl
!pip install ../input/pywheels/pydegensac-0.1.2-cp37-cp37m-linux_x86_64.whl

# install Quadree attention
!cp -r ../input/othermodels/github_QuadTreeAttention-master_renamed_pkg_src2 /kaggle/working/ # input folder is read only
!cd /kaggle/working/github_QuadTreeAttention-master_renamed_pkg_src2/QuadTreeAttention/ && pip install .
sys.path.append("/kaggle/working/github_QuadTreeAttention-master_renamed_pkg_src2")
sys.path.append("/kaggle/working/github_QuadTreeAttention-master_renamed_pkg_src2/FeatureMatching/")
sys.path.append("/kaggle/working/github_QuadTreeAttention-master_renamed_pkg_src2/QuadTreeAttention/")

In [ ]:
import os
os.chdir("/kaggle/working")

import PIL
import os
import numpy as np
import cv2
import csv
import torch
import matplotlib.pyplot as plt

import kornia
from kornia_moons.feature import *
import kornia as K
import kornia.feature as KF

import gc
import sys
import time
from PIL import Image
import random
import string

import albumentations as A

src = '/kaggle/input/image-matching-challenge-2022/'

# QTA

In [ ]:
# 初始化 QTA
from FeatureMatching.src2.config.default import get_cfg_defaults as get_cfg_defaults_qta
from FeatureMatching.src2.utils.profiler import build_profiler as build_profiler_qta
from FeatureMatching.src2.lightning.lightning_loftr import PL_LoFTR as PL_LoFTR_qta

config = get_cfg_defaults_qta() # QTA config
CROP_RATIO = 3 # QTA crop ratio
TRANSFORMS = None # QTA transforms
NB_EPOCHS = 5 # QTA epochs

# INDOOT lofrt_ds_quadtree config
config.LOFTR.MATCH_COARSE.MATCH_TYPE = 'dual_softmax' # QTA match type
config.LOFTR.MATCH_COARSE.SPARSE_SPVS = False # QTA sparse spvs
config.LOFTR.RESNETFPN.INITIAL_DIM = 128 # QTA initial dim
config.LOFTR.RESNETFPN.BLOCK_DIMS=[128, 196, 256] # QTA block dims
config.LOFTR.COARSE.D_MODEL = 256 # QTA model dim     transformer里面的维度大小，这个是需要去调整的
config.LOFTR.COARSE.BLOCK_TYPE = 'quadtree' # QTA block type
config.LOFTR.COARSE.ATTN_TYPE = 'B' # QTA attention type
config.LOFTR.COARSE.TOPKS=[32, 16, 16] # QTA topks
config.LOFTR.FINE.D_MODEL = 128 # QTA model dim
config.TRAINER.WORLD_SIZE = 1 # QTA world size
config.TRAINER.CANONICAL_BS = 32 # QTA canonical batch size
config.TRAINER.TRUE_BATCH_SIZE = 1 # QTA true batch size        学习率是可以调的
_scaling = 1 # QTA scaling
config.TRAINER.ENABLE_PLOTTING = False # QTA plotting
config.TRAINER.SCALING = _scaling # QTA scaling
config.TRAINER.TRUE_LR = 1e-3 # 1e-4 config.TRAINER.CANONICAL_LR * _scaling
config.TRAINER.WARMUP_STEP = 0 #math.floor(config.TRAINER.WARMUP_STEP / _scaling)

# lightning module
qta_max_img_size = 1024 # max image size for QTA 

qta_device = "cuda" if torch.cuda.is_available() else "cpu" # set device
disable_ckpt = True # disable checkpoint
profiler_name = None # help='options: [inference, pytorch], or leave it unset
qta_profiler = build_profiler_qta(profiler_name) # QTA profiler
qta_model = PL_LoFTR_qta(config, # QTA model
                 pretrained_ckpt= "/kaggle/working/outdoor_quadtree.ckpt", # args.ckpt_path, from scratch atm
                 profiler=qta_profiler # QTA profiler
                 )
qta_matcher = qta_model.matcher # get matcher
qta_matcher.eval() # set eval mode
qta_matcher.to(qta_device) # move to device

In [ ]:
# Image utils
def load_resized_image(fname, max_image_size):
    '''
    Load image and resize it to max_image_size 最长边 扩大至指定尺寸
    '''
    img = cv2.imread(fname) # load image
    scale = max_image_size / max(img.shape[0], img.shape[1])  # get scale
    w = int(img.shape[1] * scale) # weight
    h = int(img.shape[0] * scale) # height
    img = cv2.resize(img, (w, h)) 
    return img, scale

def scale_to_resized(mkpts0, mkpts1, scale1, scale2):
    '''
    scale to original image size because we used max_image_size
    '''
    # first point
    mkpts0[:, 0] = mkpts0[:, 0] / scale1
    mkpts0[:, 1] = mkpts0[:, 1] / scale1
    
    # second point
    mkpts1[:, 0] = mkpts1[:, 0] / scale2
    mkpts1[:, 1] = mkpts1[:, 1] / scale2
    
    return mkpts0, mkpts1

In [ ]:
# QTA Utils
def load_loftr_image_orig(fname):
    img0_raw = cv2.imread(fname, cv2.IMREAD_GRAYSCALE) # 载入图片
    img0 = torch.from_numpy(img0_raw)[None][None].cuda() / 255. # 255归一化
    return img0

def put_img_on_disk(img, output_img_tag):
    img_path_on_disk = f'/kaggle/working/{output_img_tag}.png'
    cv2.imwrite(img_path_on_disk, img) # save image
    return img_path_on_disk

def calc_divide_size_smallest(im_size, coef):
    '''
    image size 必须是 coef 的倍数
    '''
    # select size dividable by coef
    if im_size % coef == 0:
        return im_size   # 能够整除，就返回原值
    return round(((im_size / coef) + 0.5)) * coef


def add_zero_padding_two_img_same(img1, img2, div_coef=32):
    '''
    add zero padding to two images to make them same size
    '''

    img1_height, img1_width, img1_channels = img1.shape # 获取图片的高度、宽度、通道数
    img2_height, img2_width, img2_channels = img2.shape # 获取图片的高度、宽度、通道数
    
    # fit both images on canvas
    max_width = max(img1_width, img2_width) # 最大宽度
    max_height = max(img1_height, img2_height) # 最大高度
    
    # use own width and height for image with zero-padding
    result1, offset1 = create_zero_padding_img(img1, max_width, max_height, img1_channels, div_coef) # 填充图片1
    result2, offset2 = create_zero_padding_img(img2, max_width, max_height, img2_channels, div_coef) # 填充图片2
    
    return result1, result2, offset1, offset2


def create_zero_padding_img(img, max_im_width, max_im_height, channels, div_coef=32):
    # create new image of desired size and color (black) for padding
    new_area_image_width = calc_divide_size_smallest(max_im_width, div_coef) # 计算新的宽度
    new_area_image_height = calc_divide_size_smallest(max_im_height, div_coef) # 计算新的高度
    
    # it is different from what we have in max_im_width and max_im_height
    # max_im_height, max_im_width define an area for image and maybe include extra mask
    im_height, im_width, im_channels = img.shape # 获取图片的高度、宽度、通道数
    x_offset = (new_area_image_width - im_width) // 2 # 计算x轴偏移量
    y_offset = (new_area_image_height - im_height) // 2 # 计算y轴偏移量
    
    im_right  = x_offset + im_width  # right of image corner where ends 
    im_bottom = y_offset + im_height # right of image corner where ends
    
    
    color = (0,0,0) # 置零
    result = np.full((new_area_image_height, new_area_image_width, im_channels), color, dtype=np.uint8) # 创建全0图片
    
    # copy img image into center of result image
    result[y_offset:im_bottom, x_offset:im_right] = img # 将图片拷贝到中心位置
    
    # return image and x,y of old image in a new image (frame)
    return result, (x_offset, y_offset) # 返回新的图片 和 原图片的偏移量


def unpad_matches(mkpts0, mkpts1, offset_point1, offset_point2):
    '''
    remove padding from two images
    '''
    offset_x1, offset_y1 = offset_point1 # 获取图片1偏移量
    offset_x2, offset_y2 = offset_point2 # 获取图片2偏移量
    # remove points on padding
     # 去除偏移量
    mkpts0[:, 0] = mkpts0[:, 0] - offset_x1
    mkpts0[:, 1] = mkpts0[:, 1] - offset_y1
    
    mkpts1[:, 0] = mkpts1[:, 0] - offset_x2
    mkpts1[:, 1] = mkpts1[:, 1] - offset_y2
    return mkpts0, mkpts1



# QTA inference
def qta_inference(image_fpath_1, image_fpath_2, max_image_size=qta_max_img_size, divide_coef=32):
    # https://github.com/zju3dv/LoFTR/blob/master/notebooks/demo_single_pair.ipynb
    # resize image if we need
    img1_resized, scale1 = load_resized_image(image_fpath_1, max_image_size) # image1 最长边 扩大至指定尺寸
    img2_resized, scale2 = load_resized_image(image_fpath_2, max_image_size) # image2 最长边 扩大至指定尺寸
    
    ### add padding -> use same image padding mask because models wants it
    pad_img1, pad_img2, pad_offset_p1, pad_offset_p2 = add_zero_padding_two_img_same(img1_resized, img2_resized, divide_coef) # padding图片

    # save temporarily 保存新图片到磁盘    
    img1_disk_path = put_img_on_disk(pad_img1, 'qta_img1')
    img2_disk_path = put_img_on_disk(pad_img2, 'qta_img2')
    
    # 读取新图片，并做255归一化
    gray_img_1 = load_loftr_image_orig(img1_disk_path) 
    gray_img_2 = load_loftr_image_orig(img2_disk_path)
    
    batch = {'image0': gray_img_1, 'image1': gray_img_2}
    
    # Inference
    with torch.no_grad():
        qta_matcher.eval() # 取消梯度计算
        qta_matcher.to(qta_device) # 将模型放到GPU
        
        qta_matcher(batch) # 运行模型
        mkpts0 = batch['mkpts0_f'].cpu().numpy() # mkpts0
        mkpts1 = batch['mkpts1_f'].cpu().numpy() # mkpts1
    
    ### unpad matches
    mkpts0, mkpts1 = unpad_matches(mkpts0, mkpts1, pad_offset_p1, pad_offset_p2) # 去除padding
    
    ### scale to original im size because we used max_image_size
    mkpts0, mkpts1 = scale_to_resized(mkpts0, mkpts1, scale1, scale2) # 将图像缩放到原图大小
    
    # clean up 删除临时文件
    if os.path.exists(img1_disk_path): os.remove(img1_disk_path)
    if os.path.exists(img2_disk_path): os.remove(img2_disk_path) 
    
    return mkpts0, mkpts1


# LoFTR + SuperGlue + DKM

## init

In [ ]:
# LoFTR
sys.path.append('../input/othermodels/github_LoFTR-master') 

# Kornia, a dual-softmax operator
# models https://drive.google.com/drive/folders/1xu2Pq6mZT5hmFgiYMBT9Zt8h1yO-3SIp

kornia_max_image_size = 1120 # max image size for LoFTR 表示 LoFTR 算法处理的图像的最大尺寸是 1120 像素；
kornia_at_least_matches = 280 # at least matches for LoFTR 表示在 LoFTR 算法中至少需要匹配 280 个特征点，才能被认为是有效的匹配；
kornia_thrs_conf_match = 0.3 # threshold for confidence match for LoFTR 表示在 LoFTR 算法中特征点匹配的置信度阈值为 0.3；
kornia_max_matches = 1000 # -1 -> use all 表示在 LoFTR 算法中最多匹配 1000 个特征点。如果设置为 -1，则使用所有的特征点进行匹配。

kf_loftr_out_device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda') # use cpu or cuda
kf_loftr_out_matcher = KF.LoFTR(pretrained=None) # load model
kf_loftr_out_matcher.load_state_dict(torch.load("/kaggle/input/kornia-loftr/loftr_outdoor.ckpt")['state_dict']) # load weights
kf_loftr_out_matcher = kf_loftr_out_matcher.to(kf_loftr_out_device).eval() # set to eval mode

In [ ]:
# SuperGlue
sys.path.append("../input/super-glue-pretrained-network")
from models.matching import Matching
from models.utils import (AverageTimer, read_image)

sg_device = "cuda" if torch.cuda.is_available() else "cpu" # use cpu or cuda
# -1 before. -1 use original size, 1600 -> recommended size
resize = [1600, ] # resize to this size
resize_float = True # use float32

# new isamu SG tuning parameters
config = {
    "superpoint": {
        "nms_radius": 3,
        "keypoint_threshold": 0.001, 
        "max_keypoints": 1280 
    },
    "superglue": {
        "weights": "outdoor", 
        "sinkhorn_iterations": 20, 
        "match_threshold": 0.1,
    }
}
sg_matching = Matching(config).eval().to(sg_device) # load model

# filtering
use_sg_filter_strategy = False # use SG filter strategy
sg_max_confid_matches = 300 # max matches for SG
sg_thrs_conf_match = 0.3 # threshold for confidence match for SG

In [ ]:
# DKM
!mkdir -p pretrained/checkpoints
!cp ../input/othermodels/dkm_base_v11.pth pretrained/checkpoints/dkm_base_v11.pth

!pip install -f ../input/pywheels --no-index einops 
!cp -r ../input/othermodels/github_dkm-main_fix_kaggle/ /kaggle/working/DKM/

sys.path.append('/kaggle/working/DKM/')

# DKM
import torch
torch.hub.set_dir('/kaggle/working/pretrained/')
from dkm import dkm_base

 # load model
dkm_model = dkm_base(
                pretrained=True,
                version="v11",
                use_cuda=True, 
                request_im_size=(672, 896) # resize to (h, w)
                     )

dkm_ft_num = 1000 # dkm match number
dkm_max_confid_matches = 200 # max matches for confident
dkm_thrs_conf_match = 0.3 # threshold for confidence match for DKM

## Utils

In [ ]:
def load_torch_image(fname, device, max_image_size):
    '''
    Load image from file and convert to torch tensor. 最长边 扩大至指定尺寸
    '''
    img = cv2.imread(fname)
    scale = max_image_size / max(img.shape[0], img.shape[1])  # 扩大倍数
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    img = cv2.resize(img, (w, h))
    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img.to(device), scale


def filter_conf_matches(mkpts0, mkpts1, mconf, thrs_conf_match=0.2, max_matches=-1):
    '''
    Filter matches by confidence.
    '''
    # sort points by confidence descending
    mkps1_sorted = [x for (y,x) in sorted(zip(mconf,mkpts0), key=lambda pair: pair[0], reverse=True)] # 根据置信度降序排序image1的点
    mkps2_sorted = [x for (y,x) in sorted(zip(mconf,mkpts1), key=lambda pair: pair[0], reverse=True)] # 根据置信度降序排序image2的点
    
    # overwrite
    mkps1 = np.array(mkps1_sorted) 
    mkps2 = np.array(mkps2_sorted)

    num_thrsh_greater = (mconf >= thrs_conf_match).sum() # 置信度大于阈值的点的数量

    take_first_el = min(max_matches, num_thrsh_greater) # matches 数量限制
    if take_first_el > 0 and len(mkps1) > take_first_el:
        mkps1 = mkps1[:take_first_el] # image1 取前take_first_el个点
        mkps2 = mkps2[:take_first_el] # image2 取前take_first_el个点
    return mkps1, mkps2

In [ ]:
def loftr_inference(fname1, fname2, max_image_size):    
    '''
    Infer matches using LoFTR.
    '''
    image_1, scale1 = load_torch_image(fname1, kf_loftr_out_device, max_image_size) # image1 最长边 扩大至指定尺寸
    image_2, scale2 = load_torch_image(fname2, kf_loftr_out_device, max_image_size) # image2 最长边 扩大至指定尺寸
    
    # 颜色格式
    input_dict = {"image0": K.color.rgb_to_grayscale(image_1),
                  "image1": K.color.rgb_to_grayscale(image_2)
              }

    with torch.no_grad():
        correspondences = kf_loftr_out_matcher(input_dict) # LoFTR模型 推理两张图片
        
    # mkpts0, mkpts1, confidence
    mkpts0 = correspondences['keypoints0'].cpu().numpy() # keypoints0
    mkpts1 = correspondences['keypoints1'].cpu().numpy() # keypoints1
    
    # 将坐标缩放到原图
    mkpts0 = mkpts0 / scale1 
    mkpts1 = mkpts1 / scale2     
    
    return mkpts0, mkpts1


def superglue_inference(image_fpath_1, image_fpath_2):    
    image_1, inp_1, scales_1 = read_image(image_fpath_1, sg_device, resize, 0, resize_float) # 读取image1，并resize
    image_2, inp_2, scales_2 = read_image(image_fpath_2, sg_device, resize, 0, resize_float) # 读取image2，并resize
    pred = sg_matching({"image0": inp_1, "image1": inp_2}) # superglue模型 推理两张图片
    pred = {k: v[0].detach().cpu().numpy() for k, v in pred.items()} # 将tensor转为numpy
    kpts1, kpts2 = pred["keypoints0"], pred["keypoints1"] # kpts1 和 kpts2
    matches, conf = pred["matches0"], pred["matching_scores0"] # matches 和 conf

    ####
    valid = matches > -1 # 有效的点
    mkpts1 = kpts1[valid]  # mkpts1
    mkpts2 = kpts2[matches[valid]] # mkpts2
    mconf = conf[valid] # mconf
    
    # scales_1 and scales_2 are equal because we resize to max size and keep aspect ratio 
    # scales_1 is a tuple (scale1, scale2)
    scale1, scale2 = scales_1 # 扩大的倍数
    
    # because of https://github.com/magicleap/SuperGluePretrainedNetwork/blob/c0626d58c843ee0464b0fa1dd4de4059bfae0ab4/models/utils.py#L240
    mkpts1, mkpts2 = scale_to_resized(mkpts1, mkpts2, 1./scale1, 1./scale2)
        
    # filter matches by confidence
    if use_sg_filter_strategy:
        mkpts1, mkpts2 = filter_conf_matches(   
            mkpts1,
            mkpts2,
            mconf, 
            thrs_conf_match=sg_thrs_conf_match, # 0.3, 置信度阈值 
            max_matches=sg_max_confid_matches,  # 300, matches 数量限制
            )
    
    return mkpts1, mkpts2


def dkm_inference(image_fpath_1, image_fpath_2):
    # https://github.com/Parskatt/DKM
    # internally it resizes to h=384 w=512    
    img1 = cv2.imread(image_fpath_1) # 读取image1
    img2 = cv2.imread(image_fpath_2) # 读取image2

    # 将图片转为PIL格式
    img1PIL = Image.fromarray(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)) 
    img2PIL = Image.fromarray(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)) 
    
    #dense_matches, dense_certainty = dkm_model.match(img1PIL, img2PIL)
    dense_matches, dense_certainty = dkm_model.match(img1PIL, img2PIL, use_cuda=True) # 模型推理, 使用cuda
    
    # # You may want to process these, e.g. we found dense_certainty = dense_certainty.sqrt() to work quite well in some cases.
    dense_certainty = dense_certainty.sqrt() # 将置信度开根号
    # Sample 10000 sparse matches
    sparse_matches, sparse_certainty = dkm_model.sample(dense_matches, dense_certainty, dkm_ft_num) ####

    mkps1 = sparse_matches[:, :2] # mkps1
    mkps2 = sparse_matches[:, 2:] # mkps2

    ####
    h, w, c = img1.shape # 
    mkps1[:, 0] = ((mkps1[:, 0] + 1)/2) * w
    mkps1[:, 1] = ((mkps1[:, 1] + 1)/2) * h

    h, w, c = img2.shape
    mkps2[:, 0] = ((mkps2[:, 0] + 1)/2) * w
    mkps2[:, 1] = ((mkps2[:, 1] + 1)/2) * h
    
    
    # filter matches by confidence
    mkps1, mkps2 = filter_conf_matches(
        mkps1,  # mkps1
        mkps2,  # mkps2
        sparse_certainty,  # mconf
        thrs_conf_match=dkm_thrs_conf_match, # 0.3, 置信度阈值
        max_matches=dkm_max_confid_matches,  # 200, matches 数量限制
        )
    
    return mkps1, mkps2, sparse_certainty


# Inference

In [ ]:
def inf_models_ensemble(image_fpath_1, image_fpath_2):
    '''
    Input : image1 和 image2, 进行inference ensemble
    Return: match keypoints (匹配关键点)
    '''
    # loftr
    k_mkpts1, k_mkpts2 = loftr_inference(image_fpath_1, image_fpath_2, kornia_max_image_size) # kornia_max_image_size = 1120
    mkpts1_merge = k_mkpts1
    mkpts2_merge = k_mkpts2
   
    #  QuadTreeAttention
    qta_mkps1,qta_mkps2 = qta_inference(image_fpath_1, image_fpath_2, max_image_size=qta_max_img_size, divide_coef=32) # qta_max_img_size = 1024
    mkpts1_merge = np.concatenate((mkpts1_merge, qta_mkps1), axis=0)
    mkpts2_merge = np.concatenate((mkpts2_merge, qta_mkps2), axis=0)
    
    # SUPERGLUE
    sg_mkpts1, sg_mkpts2 = superglue_inference(image_fpath_1, image_fpath_2) 
    mkpts1_merge = np.concatenate((mkpts1_merge, sg_mkpts1), axis=0)
    mkpts2_merge = np.concatenate((mkpts2_merge, sg_mkpts2), axis=0)

    # DKM
    dkm_mkpts1, dkm_mkpts2, _ = dkm_inference(image_fpath_1, image_fpath_2)
    if len(dkm_mkpts1) > 0:
        mkpts1_merge = np.concatenate((mkpts1_merge, dkm_mkpts1), axis=0)
        mkpts2_merge = np.concatenate((mkpts2_merge, dkm_mkpts2), axis=0)

    return mkpts1_merge, mkpts2_merge # get match keypoints


def calc_F_matrix(item):
    '''
    Input : sample_id, mkps1, mkps2 # 一对匹配关键点
    Return: F-matrix
    '''
    sample_id, mkps1, mkps2 = item 

    if len(mkps1) > 7: # 如果有足够的点, 则进行F-matrix计算
        F, _ = cv2.findFundamentalMat(mkps1, mkps2, cv2.USAC_MAGSAC, ransacReprojThreshold=0.25, confidence=0.999999, maxIters=120_000)  # 计算F-matrix
    else:   # 如果没有足够的点, 则返回空的F-matrix
        F = None   

    del mkps1, mkps2 # clean up
    return sample_id, F

In [ ]:
test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]
        
test_samples_run_optim = test_samples

In [ ]:
# 导入线程池
from concurrent.futures import ThreadPoolExecutor 
from concurrent import futures
F_dict = {} # F-matrix
prev_matches_item = None # previous matches item

for i, (sample_id, batch_id, image_1_id, image_2_id) in enumerate(test_samples_run_optim):
    image_fpath_1 = f'{src}/test_images/{batch_id}/{image_1_id}.png' # image1 path
    image_fpath_2 = f'{src}/test_images/{batch_id}/{image_2_id}.png' # image2 path
    
    if prev_matches_item is None:
        # 初始化 只获取匹配关键点
        mkpts1_merge, mkpts2_merge = inf_models_ensemble(image_fpath_1, image_fpath_2) # get match keypoints
        prev_matches_item = (sample_id, mkpts1_merge, mkpts2_merge) # update prev_matches_item
    else:
        # calc F matrix and matches in parallel
        with ThreadPoolExecutor(max_workers=2) as executor:
            future1 = executor.submit(inf_models_ensemble, image_fpath_1=image_fpath_1, image_fpath_2=image_fpath_2) # 任务1: inf_models_ensemble, 获取匹配关键点
            future2 = executor.submit(calc_F_matrix, item=prev_matches_item) # 任务2: calc_F_matrix, 计算上一对匹配关键点的F-matrix
            future_list = [future1,future2]

            finished, pending = futures.wait(
                future_list, # 待完成的任务列表
                return_when=futures.ALL_COMPLETED # 当所有任务完成时, 返回结果
                )
            
            mkpts1_merge, mkpts2_merge = future1.result()
            old_sample_id, F = future2.result()

            # results are ready
            F_dict[old_sample_id] = np.zeros((3, 3)) if F is None else F  # 存入F_dict
            prev_matches_item = (sample_id, mkpts1_merge, mkpts2_merge) # update prev_matches_item
    
    try:
        # clean up
        gc.collect()
        torch.cuda.empty_cache()
    except Exception:
        pass

# process last item
if prev_matches_item is not None:
    sample_id, F = calc_F_matrix(prev_matches_item) # 计算上一对匹配关键点的F-matrix
    F_dict[sample_id] = np.zeros((3, 3)) if F is None else F  # 存入F_dict

In [ ]:
def FlattenMatrix(M, num_digits=8):
    # Matrix to String
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

# 写入 submission.csv
with open('submission.csv', 'w') as f:
    f.write('sample_id,fundamental_matrix\n')
    for sample_id, F in F_dict.items():
        f.write(f'{sample_id},{FlattenMatrix(F)}\n')